In [72]:
import json
import glob
import csv
import bs4
import os
import pandas as pd
import pandas_datareader as pdr
import numpy as np
from selenium import webdriver
import yfinance as yf
import time
import scipy
import datetime
from IPython.display import display

In [31]:
DFList:list=[pd.DataFrame] * 5;
#perform arithmetic operations on date
#print(datetime.datetime.today().strftime('%d-%m-%Y')-datetime.timedelta(days=1));
print(type(datetime.datetime.today()-datetime.timedelta(days=2)));
DateConvert_Str:datetime.datetime=datetime.datetime.today()-datetime.timedelta(days=1);
DateString:str=datetime.datetime.strftime(DateConvert_Str,'%d-%m-%Y');
ThaiSS_FileStr:str='/Users/cedricyang/Downloads/Short Sales_'+DateString+' to '+DateString+'.xlsx';
print(ThaiSS_FileStr);
#or implement checks to see if the files was downloaded successfully
DirListFile:list=os.listdir('/Users/cedricyang/Downloads/');
List_of_file:list=glob.glob('/Users/cedricyang/Downloads/*.xlsx');
print(List_of_file)
NameString:str='/Users/cedricyang/Downloads/';

<class 'datetime.datetime'>
/Users/cedricyang/Downloads/Short Sales_12-04-2023 to 12-04-2023.xlsx
['/Users/cedricyang/Downloads/data_4257_20230411.xlsx', '/Users/cedricyang/Downloads/Short Sales_10-04-2023 to 10-04-2023.xlsx', '/Users/cedricyang/Downloads/Short Sales_12-04-2023 to 12-04-2023.xlsx', '/Users/cedricyang/Downloads/Equities_9988.xlsx', '/Users/cedricyang/Downloads/Short Sales_11-04-2023 to 11-04-2023.xlsx', '/Users/cedricyang/Downloads/Short Sales_07-04-2023 to 07-04-2023.xlsx', '/Users/cedricyang/Downloads/Short Sales_12-04-2023 to 12-04-2023(1).xlsx', '/Users/cedricyang/Downloads/data_3431_20230403.xlsx', '/Users/cedricyang/Downloads/Short Sales_10-04-2023 to 10-04-2023(1).xlsx']


In [34]:
FileIO=open('IRSG.json','r');
DictObjs:dict=json.load(FileIO);
DFList[0]:pd.DataFrame=pd.json_normalize(DictObjs,record_path=['result','records']);
Sora_Quantile=DFList[0]['comp_sora_3m'].quantile([0.25,0.5,0.75,0.95,1.0]).to_dict();
DFList[0]['Indicator 3M > 1M']=DFList[0]['comp_sora_3m']>DFList[0]['comp_sora_1m'];
DFList[0]['Indicator 1M > 3M']=DFList[0]['comp_sora_1m']>DFList[0]['comp_sora_3m'];
display(DFList[0])

end_of_day  comp_sora_1m  comp_sora_3m  comp_sora_6m  aggregate_volume  \
0    2023-04-13           NaN           NaN           NaN               NaN   
1    2023-04-12        3.5939        3.6106        3.3213            1726.0   
2    2023-04-11        3.6116        3.6123        3.3192            2554.0   
3    2023-04-10        3.6302        3.6127        3.3202            1288.0   
4    2023-04-06        3.6497        3.6156        3.3189            1086.0   
..          ...           ...           ...           ...               ...   
495  2021-04-23        0.1457        0.1864        0.1771            1680.0   
496  2021-04-22        0.1501        0.1844        0.1759            1813.0   
497  2021-04-21        0.1488        0.1840        0.1755            2150.0   
498  2021-04-20        0.1514        0.1841        0.1752            1577.0   
499  2021-04-19        0.1529        0.1835        0.1750            2070.0   

      timestamp  Indicator 3M > 1M  Indicator 1M > 3M  
0    1681384061              False              False  
1    1681384061               True              False  
2    1681384061               True              False  
3    1681384061              False               True  
4    1681384061              False               True  
..          ...                ...                ...  
495  1681384061               True              False  
496  1681384061               True              False  
497  1681384061               True              False  
498  1681384061               True              False  
499  1681384061               True              False  

[500 rows x 8 columns]

In [4]:
DriverObjs:webdriver.firefox.webdriver.WebDriver=webdriver.Firefox();
DriverObjs.get('https://www.set.or.th/en/market/statistics/short-sell');
time.sleep(2);
print(DriverObjs.title);
DriverObjs.find_element(webdriver.common.by.By.XPATH,'//*[@id=\"__layout\"]/div/div[2]/div[3]/div/div/div[2]/div/div[2]/button').click();
pass; #similar to __asm__("nop")
time.sleep(2);
DriverObjs.close();

Short Selling - The Stock Exchange of Thailand


In [5]:
FileString:str='/Users/cedricyang/Downloads/Short Sales_10-04-2023 to 10-04-2023.xlsx';
ThaiSS_DF:pd.DataFrame=pd.read_excel(io=ThaiSS_FileStr,engine='openpyxl',header=5,index_col=0);
ThaiSS_DF=ThaiSS_DF[:200];
ThaiSS_DF=ThaiSS_DF.sort_values(by=['Turnover (Baht)','Symbol'],ascending=False)
ThaiSS_DF=ThaiSS_DF.reset_index();
ThaiSS_DF['USD Eqiv Amt']=(ThaiSS_DF['Turnover (Baht)'] / 34.12);
display(ThaiSS_DF[:30])

Symbol  Volume (Shares)  Turnover (Baht)  \
0      BANPU       30084700.0      299175815.0   
1       BDMS        7343600.0      220934625.0   
2        AOT        1891800.0      136560675.0   
3    BANPU-R       13132700.0      130260205.0   
4     ADVANC         600600.0      127475300.0   
5        KTB        7348000.0      126263570.0   
6      CPALL        1976100.0      122118650.0   
7      KBANK         895400.0      120912800.0   
8    KBANK-R         797400.0      107636950.0   
9      CBG-R        1140300.0       98163775.0   
10     AOT-R        1303500.0       94045775.0   
11       CPF        4195100.0       86658350.0   
12       BBL         514800.0       80853850.0   
13     KTB-R        4718200.0       80804130.0   
14      GULF        1252100.0       67613400.0   
15     CPF-R        2934400.0       60632650.0   
16       IVL        1514800.0       51192350.0   
17      BH-R         187800.0       44162600.0   
18     CRC-R         887500.0       39637200.0   
19    COM7-R        1253500.0       34248750.0   
20        LH        3341500.0       32847755.0   
21     BGRIM         810200.0       32322875.0   
22   CPALL-R         503400.0       31157900.0   
23      COM7        1117000.0       30432575.0   
24     JMT-R         707800.0       30318175.0   
25     KKP-R         472500.0       30041150.0   
26       BCP         905500.0       28869625.0   
27     BCH-R        1291000.0       27471300.0   
28       BEM        3002500.0       26872385.0   
29  ADVANC-R         121600.0       25815200.0   

    % Short Sale Volume Comparing with Auto Matching  USD Eqiv Amt  
0                                           0.174635  8.768342e+06  
1                                           0.110063  6.475223e+06  
2                                           0.086170  4.002364e+06  
3                                           0.076232  3.817708e+06  
4                                           0.158625  3.736087e+06  
5                                           0.119802  3.700574e+06  
6                                           0.124498  3.579093e+06  
7                                           0.047159  3.543751e+06  
8                                           0.041997  3.154659e+06  
9                                           0.212893  2.877016e+06  
10                                          0.059373  2.756324e+06  
11                                          0.205662  2.539811e+06  
12                                          0.114207  2.369691e+06  
13                                          0.076926  2.368234e+06  
14                                          0.130443  1.981635e+06  
15                                          0.143857  1.777041e+06  
16                                          0.128057  1.500362e+06  
17                                          0.072952  1.294332e+06  
18                                          0.104458  1.161700e+06  
19                                          0.057367  1.003773e+06  
20                                          0.288930  9.627126e+05  
21                                          0.085224  9.473293e+05  
22                                          0.031715  9.131858e+05  
23                                          0.051120  8.919278e+05  
24                                          0.095333  8.885749e+05  
25                                          0.129808  8.804557e+05  
26                                          0.200901  8.461203e+05  
27                                          0.119688  8.051377e+05  
28                                          0.146660  7.875846e+05  
29                                          0.032116  7.566002e+05

In [6]:
DriverObjs2:webdriver.firefox.webdriver.WebDriver=webdriver.Firefox();
DriverObjs2.get('https://www.set.or.th/en/home');
time.sleep(2);
DriverObjs2.find_element(webdriver.common.by.By.XPATH,'/html/body/div/div/div/header/div[1]/div[1]/div[1]/div[3]/div/div[1]/input').click();
time.sleep(2);
TrendingQuote=DriverObjs2.find_element(webdriver.common.by.By.XPATH,'//*[@id="SearchDropdownSecurities"]');
print(TrendingQuote.text);

Popular Quote
Last Update 13 Apr 2023 09:56:54
BANPU
Stock
9.95
BANPU PUBLIC COMPANY LIMITED
-0.05 (-0.50%)
SCB
Stock
105.50
SCB X PUBLIC COMPANY LIMITED
-0.50 (-0.47%)
PTT
Stock
31.50
PTT PUBLIC COMPANY LIMITED
0.00 (0.00%)
KBANK
Stock
136.00
KASIKORNBANK PUBLIC COMPANY LIMITED
+1.00 (+0.74%)
TISCO
Stock
101.50
TISCO FINANCIAL GROUP PUBLIC COMPANY LIMITED
+0.50 (+0.50%)


In [17]:
#creating a list of list
ListLoop:list=[list]*630;
print(ListLoop);

[<class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 'list'>, <class 

In [18]:
HtmlIO_File=open('HK_X_SS.html');
soupObj=bs4.BeautifulSoup(HtmlIO_File,'html.parser');
preTags:str=soupObj.select_one('pre').text;
SplitList:list=preTags.split('\n')[5:];
print(len(SplitList));

628


In [86]:
jjj:int=0;
List_indexer:int=0
for i in range(4,608):
    ListLoop[List_indexer]=SplitList[i].split(" ");
    ListLoop[List_indexer]=[x for x in ListLoop[List_indexer] if x != '']
    List_indexer=List_indexer+1;
    #x for x in list_with_empty_strings if x != ''

LoopCounters:int=0;
OuterList_len:int=len(ListLoop);
#Testing Relational Operator
print(ListLoop[OuterList_len-29][0]);
print(len(ListLoop[OuterList_len-29][0]));
print(ListLoop[OuterList_len-29][0]!='%82800');
print(ListLoop[OuterList_len-29][1]!='TRACKER');
    
while (ListLoop[LoopCounters][0]!='%82800'):
    Each_List_len=len(ListLoop[LoopCounters]);
    if (ListLoop[LoopCounters][0]=='%'): #indicates it's a ETF
        ListLoop[LoopCounters][Each_List_len-Each_List_len]='ETF';
        ListLoop[LoopCounters][Each_List_len-1]=int(ListLoop[LoopCounters][Each_List_len-1].replace(',',''));
        ListLoop[LoopCounters][Each_List_len-2]=int(ListLoop[LoopCounters][Each_List_len-2].replace(',',''))
        LoopCounters=LoopCounters+1;
        continue;
    #Stored the converted value back to memory.. slow CPU operations
    ListLoop[LoopCounters][Each_List_len-Each_List_len]=int(ListLoop[LoopCounters][Each_List_len-Each_List_len]);
    ListLoop[LoopCounters][Each_List_len-1]=int(ListLoop[LoopCounters][Each_List_len-1].replace(',',''));
    ListLoop[LoopCounters][Each_List_len-2]=int(ListLoop[LoopCounters][Each_List_len-2].replace(',',''));
    LoopCounters=LoopCounters+1;
    continue;

%82800
6
False
False


In [88]:
with open("ListofList.csv", "w", newline="\n") as f:
    writer = csv.writer(f,delimiter=',',quoting=csv.QUOTE_MINIMAL);
    writer.writerow(ListLoop);

SyntaxError: invalid syntax (3006935689.py, line 2)